<a href="https://colab.research.google.com/github/mkaramb/CloudWeaver/blob/retriever-chain-connection/DB_Connected_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
!pip install -U --upgrade --quiet langchain-google-vertexai langchain-google-genai langchain-core langchain-community langchain unstructured lark chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 817.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

os.environ['GOOGLE_API_KEY'] = 'KEY'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

# Access Database contents and metadata

In [ ]:
file_path = '/content/drive/My Drive/cw_db_metadata.pkl'

In [ ]:
import pickle

# Load the file
with open(file_path, 'rb') as file:
    documents = pickle.load(file)

In [ ]:
print(documents[1356].page_content)


/******************************************
	Routes
 *****************************************/
resource "google_compute_route" "route" {
  provider = google-beta
  count    = var.routes_count

  project = var.project_id
  network = var.network_name

  name                   = lookup(var.routes[count.index], "name", format("%s-%s-%d", lower(var.network_name), "route", count.index))
  description            = lookup(var.routes[count.index], "description", null)
  tags                   = compact(split(",", lookup(var.routes[count.index], "tags", "")))
  dest_range             = lookup(var.routes[count.index], "destination_range", null)
  next_hop_gateway       = lookup(var.routes[count.index], "next_hop_internet", "false") == "true" ? "default-internet-gateway" : null
  next_hop_ip            = lookup(var.routes[count.index], "next_hop_ip", null)
  next_hop_instance      = lookup(var.routes[count.index], "next_hop_instance", null)
  next_hop_instance_zone = lookup(var.routes[count.inde

# Retriever

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata

doc_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_document"
)

# Assuming you've already filtered 'documents' to exclude those with empty 'page_content'
# as shown in the previous code snippet:
filtered_documents = [doc for doc in documents if doc.page_content and doc.page_content.strip()]

# Assuming 'filtered_documents' is your list of Document objects ready for processing
# Apply the filter to clean up metadata in your documents
cleaned_documents = filter_complex_metadata(filtered_documents)

# Now that the documents have been cleaned, you can proceed with creating the vector store
vectorstore = Chroma.from_documents(documents=cleaned_documents, embedding=doc_embeddings)


In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="sub_instance",
        description="Specifies the exact variant or configuration of the instance that the Terraform code represents. This allows for precise identification of Terraform files based on specific implementations, such as a MySQL version for a Cloud SQL instance, enabling targeted retrieval of code.",
        type="string",
    ),
    AttributeInfo(
        name="product",
        description="Identifies the broader GCP product category to which an instance belongs. For example, a Compute Engine VM or a Cloud SQL database would fall under 'compute' and 'sql' resources, respectively. This categorization facilitates the organization and search of Terraform files within the context of GCP products.",
        type="string",
    ),
    AttributeInfo(
        name="instance",
        description="Denotes the specific instance within a GCP product that the Terraform code is designed to provision or manage. This could refer to a particular VM, database, or storage bucket, among others. The instance name aids in pinpointing Terraform files that apply to particular GCP service instances.",
        type="string",
    ),
    AttributeInfo(
        name="folder_type",
        description="""Modules folder_type:
This directory contains Terraform modules, which are pre-packaged configurations for specific use cases. Each module comprises resource and module blocks that encapsulate standard configurations for GCP products or functionalities. These modules are designed for reuse across different parts of the project or in other projects, promoting efficient and scalable infrastructure setups.

Examples folder_type:
The examples directory showcases how to use the Terraform modules from the modules directory in practical deployments. It includes sample Terraform configurations that reference and instantiate the modules, providing clear, real-world scenarios of module application. This directory is instrumental in demonstrating the modules' utility and easing their adoption by offering ready-to-use examples. """,
        type="string",
    ),
    AttributeInfo(
        name="type",
        description="""In a Terraform project aimed at deploying and managing resources on the Google Cloud Platform (GCP), three critical files play unique roles in ensuring the configuration's flexibility, clarity, and operational efficiency. These files—`variables.tf`, `main.tf`, and `outputs.tf`—each serve distinct purposes within the Terraform infrastructure as code (IaC) framework.

Variables.tf:

The `variables.tf` file is fundamental for defining and organizing variables to customize infrastructure deployments across various environments. Each variable is declared with a name, type, description, and optionally, a default value. This structure not only aids in making Terraform configurations modular and reusable but also enhances the maintainability of the code. Variables range from simple placeholders (e.g., project IDs, geographical locations) to complex structures for detailed resource definitions, embodying the principles of IaC by promoting a systematic and customizable approach to cloud resource deployment.

Main.tf:

At the core of a Terraform project, the `main.tf` file orchestrates the deployment of GCP resources. It integrates:

- **Local Variables (`locals`)** to centralize configurations, reducing repetition and enhancing configuration clarity.
- **Resource Blocks** to define the GCP resources to be managed or provisioned, including IAM roles and storage buckets.
- **Modules** to encapsulate and reuse configurations for efficient resource management.
- **Data Sources (`data`)** to incorporate data from GCP or external sources for informed configuration steps.
- **IAM and Permissions** to detail the setup for service accounts and specify roles for secure access management.

This file exemplifies the declarative nature of IaC, showcasing Terraform's capability to manage a diverse array of resources on GCP through a structured and scalable configuration.

Outputs.tf:

The `outputs.tf` file is dedicated to defining output variables that relay crucial deployment information to the user. These outputs highlight essential details such as resource identifiers, names, descriptions, and project-specific information, bridging the gap between complex configurations and actionable insights. By making critical deployment information accessible, the outputs.tf file not only enhances the transparency of cloud resources but also supports further automation, integration, and management tasks, emphasizing the importance of information accessibility in IaC practices.

README.md:

The README.md file serves as the project's documentation hub, containing critical information on how to use the Terraform product, module, or example. It outlines expected inputs, outputs, dependencies, requirements, and use cases, offering a comprehensive guide to navigating and utilizing the Terraform configuration effectively. This file ensures that users have a clear understanding of the project's scope, functionalities, and operational requirements, making it an indispensable resource for successful deployment.
Together, these files provide a solid framework for managing infrastructure on GCP with Terraform, leveraging the full potential of IaC to deliver adaptable, scalable, and efficiently managed cloud resources, complemented by thorough documentation for user guidance and project clarity.

Together, these files constitute a robust framework for managing infrastructure on GCP with Terraform, leveraging the strengths of IaC to deliver adaptable, scalable, and efficient cloud resource management. """,
        type="string",
    ),
]


In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever

document_content_description = "Examples of different components of GCP terraform instances and modules along with readmes describing them."
retriever = SelfQueryRetriever.from_llm(
    model,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
)

# Chain

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define the initial prompt template for generating project architecture
architecture_prompt_template = """Given the project description below, generate an initial architecture outline suitable for GCP using Terraform. Keep it concise and to the point.

Project Description:
{project_description}

User Architecture Feedback:
{user_architecture_feedback}

Initial Architecture Outline:"""

# Define the prompt template for generating Terraform code based on confirmed architecture
terraform_prompt_template = """
Given the confirmed architecture outline below, generate the full Terraform code necessary to deploy the project on GCP. The generated code should reflect the unique requirements of the specified architecture, focusing on creating a scalable and secure infrastructure. Use the example below as a guide for the format and structure but develop your own resource configurations and names based on the project's needs. Your code should include a Compute Engine instance, appropriate networking setup, and any other resources deemed necessary for a basic web application.

Note: The example is for illustration only. Please generate custom configurations based on the architecture outline.

Example Terraform Code:
resource "google_compute_instance" "default" {{
  name         = "example-instance"
  machine_type = "e2-medium"
  zone         = "us-central1-a"

  boot_disk {{
    initialize_params {{
      image = "debian-cloud/debian-9"
    }}
  }}

  network_interface {{
    network = "default"
    access_config {{}}
  }}
}}

Confirmed Architecture Outline:
{{confirmed_architecture}}

Existing Terraform Code:
{{existing_terraform_code}}

User Feedback:
{{user_terraform_feedback}}

Your Terraform Code:
"""


# Custom prompt templates
architecture_prompt = PromptTemplate.from_template(
    architecture_prompt_template)
terraform_code_prompt = PromptTemplate.from_template(
    terraform_prompt_template)

# Define the interaction chain for architecture generation and confirmation
architecture_chain = ({
    "project_description": RunnablePassthrough(),
    "user_architecture_feedback": RunnablePassthrough()
}
    | architecture_prompt
    | model
    | StrOutputParser())

# Define the interaction chain for Terraform code generation
terraform_code_chain = ({
    "context": retriever,
    "confirmed_architecture": RunnablePassthrough(),
    "existing_terraform_code": RunnablePassthrough(),
    "user_terraform_feedback": RunnablePassthrough()
}
    | terraform_code_prompt
    | model
    | StrOutputParser())

# Define the prompt template for suggesting improvements
improvement_prompt_template = """
Given the confirmed architecture outline below, suggest potential improvements that could enhance the project's efficiency, scalability, or reliability.

Confirmed Architecture Outline:
{confirmed_architecture}

Suggested Improvements:"""

improvement_prompt = PromptTemplate.from_template(
    improvement_prompt_template)

# Define the interaction chain for improvement suggestions
improvement_chain = ({
    "context": retriever,
    "confirmed_architecture": RunnablePassthrough()
}
    | improvement_prompt
    | model
    | StrOutputParser())

project_description = input("Enter the project description: ")

user_architecture_feedback = "Initial Feedback (None)"
user_terraform_feedback = "Initial Feedback (None)"
existing_terraform_code = ""

architecture = None
while not architecture:
  try:
    architecture = architecture_chain.invoke({
        "project_description": project_description,
        "user_architecture_feedback": "user_architecture_feedback"
    })
  except Exception as e:
    print("Error:", e)
    break

  print("Generated Architecture: ", architecture)
  user_confirmation = input("User confirms this architecture? (yes/no): ")
  if user_confirmation.lower() == 'yes':
    break
  else:
    user_architecture_feedback = input("Please describe the issues you have with the architecture: ")

improvements = improvement_chain.invoke(
  {"confirmed_architecture": architecture})

# Display the suggested improvements to the user
print("Suggested Improvements: ", improvements)
user_confirmation = input(
  "Would you like to incorporate suggested improvements? (yes/no): ")

if user_confirmation.lower() == 'yes':
    # Update the architecture with the improvements
    architecture += "\n\nIncorporated Improvements:\n" + improvements

# print()
# print(architecture)
terraform_code_correct = False
terraform_code_generated = None
while terraform_code_correct == False:
  print("Architecture: ", architecture)
  try:
    terraform_code_generated = terraform_code_chain.invoke({
        "confirmed_architecture": architecture,
        "existing_terraform_code": existing_terraform_code,
        "user_terraform_feedback": user_terraform_feedback
    })
  except Exception as e:
    print("Error:", e)
    break

  print("Generated Terraform Code: ", terraform_code_generated)
  user_confirmed_terraform_code = input("User confirms this Terraform code? (yes/no): ")
  if user_confirmed_terraform_code.lower() == 'yes':
    terraform_code_correct = True
  else:
      existing_terraform_code = terraform_code_generated
      user_terraform_feedback = input("What is wrong with the code? ")




Enter the project description: Create a VM instance and connect it to a sql server so that I can display documents in the sql server from the vm.
Generated Architecture:  **Compute Engine**
- VM instance running a web server
- Instance template for automated instance creation

**Cloud SQL**
- SQL Server instance
- Database for storing documents

**VPC Network**
- VPC network for connecting the VM and SQL instance
- Subnets for the VM and SQL instance
- Firewall rules to allow traffic between the VM and SQL instance

**Infrastructure as Code (IaC)**
- Terraform configuration to manage the infrastructure resources
User confirms this architecture? (yes/no): yes
Suggested Improvements:  **Efficiency Improvements:**

* **Use Cloud Load Balancing:** Distribute incoming traffic across multiple VM instances to improve responsiveness and scalability.
* **Implement Caching:** Cache frequently accessed data or pages on the web server to reduce database load and improve performance.
* **Optimize D